# Ejercicios

Construye un modelo para el dataset de mundocine y súbelo a HuggingFace. Incluye en la siguiente celda el enlace a tu modelo en HuggingFace y guarda los cambios en GitHub.  


Enlace al modelo: [sanar085/text-classification](https://huggingface.co/sanar085/text-classification)

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("muchocine")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for muchocine contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/muchocine
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/3872 [00:00<?, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating'],
        num_rows: 3872
    })
})

In [ ]:
raw_dataset['train']

Dataset({
    features: ['review_body', 'review_summary', 'star_rating'],
    num_rows: 3872
})

In [ ]:
raw_dataset['train'].to_pandas()

,review_body,review_summary,star_rating
0,"""May, ¿Quieres ser mi amigo?"" es una de esas p...","May, ¿quieres ser mi amigo?",3
1,Es todo un alivio que ante tanta película que ...,Cómo ponerse en la piel de un kamikaze,3
2,"Una fiesta llena de excesos, rubias despampana...","Silicona, esteroides, pactos demoníacos y otra...",0
3,"Zoom nos cuenta la historia de Jack Shepard, a...",Una comedia entretenida y poca cosa más para v...,1
4,Luc Besson dirige esta película basada en sus ...,"Luc Besson sabe manejar la acción, y aquí lo d...",3
...,...,...,...
3867,Hoy voy de rollo cultural y os diré que esta p...,"Película dirigida a un público adolescente, co...",2
3868,"El día que me dispuse a ver ""El Manantial de l...",Bergman en estado puro. Duro a la par que sens...,3
3869,"No es que no me guste, pero reconozco que siem...",Por fin vemos un Almodóvar que retrata escenas...,3
3870,"Tony Scott es, en mi opinión, un correcto dire...",Dos horas entretenidillas de thriller con cier...,1


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "mrm8488/electricidad-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example["review_summary"], truncation=True)

In [ ]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/3872 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3872
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column('star_rating','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3872
    })
})

In [ ]:
from datasets import DatasetDict,Dataset
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
len_dataset = len(tokenized_dataset["train"])
train_dataset = tokenized_dataset["train"][0:int(len_dataset*0.8)]
test_dataset = tokenized_dataset["train"][int(len_dataset*0.8):]
new_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("text-classification",evaluation_strategy="epoch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at mrm8488/electricidad-base-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=new_dataset["train"],
    eval_dataset=new_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.339993,0.405161
2,1.388600,1.287805,0.438710
3,0.960800,1.419749,0.450323


TrainOutput(global_step=1164, training_loss=1.1215348522278041, metrics={'train_runtime': 169.987, 'train_samples_per_second': 54.657, 'train_steps_per_second': 6.848, 'total_flos': 308308169529570.0, 'train_loss': 1.1215348522278041, 'epoch': 3.0})

In [ ]:
%cd text-classification
trainer.push_to_hub(commit_message="Training complete", tags="classification")

/content/text-classification


training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/sanar085/text-classification/commit/3bf2d05ca202a14fee056e4feb7aa8dbe652ae35', commit_message='Training complete', commit_description='', oid='3bf2d05ca202a14fee056e4feb7aa8dbe652ae35', pr_url=None, pr_revision=None, pr_num=None)